In [3]:
import pandas as pd
from sklearn.metrics import accuracy_score

import numpy as np
import os
from sklearn import linear_model

import matplotlib.pyplot as plt

import seaborn as sns

from tqdm import tqdm
from tqdm import tqdm_notebook

from scipy.signal import hilbert
from scipy.signal import hann
from scipy.signal import convolve
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
from sklearn.svm import NuSVR
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

In [ ]:
train = pd.read_csv('train.csv', dtype={'acoustic_data': np.int16, 'time_to_failure': np.float64})

In [ ]:
print('Shape is ', train.shape)
train.isnull().sum()

In [ ]:
train.keys()

In [ ]:
pd.options.display.precision = 15

In [ ]:
#tyring to make a training dataset of 150000 sampled rows.
#So we are dividing the train into segments of our total length by 
#the number of rows i.e we are sampling a part of train set.rows = 150_000

rows = 150_000
segments = int(np.floor(train.shape[0] / rows))

#This "statistical features" method can be improved by replacing 4 features 
#["ave", "std", "min", "max"] with 103 features ["ave", "std", "x0", "x1", "x2", …, "x98", "x99", "x100"], 
#where "x5" denotes the maximum value among all values covering 5% of the population.

X_train = pd.DataFrame(index=range(segments), dtype=np.float64,
                       columns=['ave', 'std', 'max', 'min','10th_per','20th_per','30th_per','40th_per','50th_per',
                                '60th_per','70th_per','80th_per','90th_per','100th_per'])
y_train = pd.DataFrame(index=range(segments), dtype=np.float64,
                       columns=['time_to_failure'])

#why this: [segment * rows : segment * rows + rows] ?

#grouping samples. Then for each group the diverse features, maximum, standard deviation, 
#and so on, are computed. The grouping is carried out to be able to compute these features 

for segment in tqdm(range(segments)):
    seg = train.iloc[segment*rows:segment*rows+rows]
    x = seg['acoustic_data'].values
    y = seg['time_to_failure'].values[-1]
    
    y_train.loc[segment, 'time_to_failure'] = y
    
    X_train.loc[segment, 'ave'] = x.mean()
    X_train.loc[segment, 'std'] = x.std()
    X_train.loc[segment, 'max'] = x.max()
    X_train.loc[segment, 'min'] = x.min()
    X_train.loc[segment, '10th_per'] = np.percentile(x, 10)
    X_train.loc[segment, '20th_per'] = np.percentile(x, 20)
    X_train.loc[segment, '30th_per'] = np.percentile(x, 30)
    X_train.loc[segment, '40th_per'] = np.percentile(x, 40)
    X_train.loc[segment, '50th_per'] = np.percentile(x, 50)
    X_train.loc[segment, '60th_per'] = np.percentile(x, 60)
    X_train.loc[segment, '70th_per'] = np.percentile(x, 70)
    X_train.loc[segment, '80th_per'] = np.percentile(x, 80)
    X_train.loc[segment, '90th_per'] = np.percentile(x, 90)
    X_train.loc[segment, '100th_per'] = np.percentile(x, 100)


In [ ]:
scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)

In [ ]:
import sklearn.linear_model
from sklearn.linear_model import LogisticRegression

In [ ]:
# Simple Linear Model

In [ ]:
lm = linear_model.LinearRegression()
model = lm.fit(X_train_scaled,y_train)

In [ ]:
predictions = lm.predict(X_train_scaled)
print(predictions)

In [ ]:
score = mean_absolute_error(y_train.values, predictions)
print(f'Score: {score:0.3f}')

In [ ]:
lm.score(X_train,y_train)

In [ ]:
lm.coef_

In [ ]:
lm.intercept_

In [ ]:
submission = pd.read_csv('sample_submission.csv', index_col='seg_id')

In [ ]:
X_test = pd.DataFrame(columns=X_train.columns, dtype=np.float64, index=submission.index)

In [ ]:
print("There are {} files in test folder".format(len(os.listdir('test' ))))

In [ ]:
for seg_id in X_test.index:
    seg = pd.read_csv('test/' + seg_id + '.csv')
    
    x = seg['acoustic_data'].values
    
    X_test.loc[seg_id, 'ave'] = x.mean()
    X_test.loc[seg_id, 'std'] = x.std()
    X_test.loc[seg_id, 'max'] = x.max()
    X_test.loc[seg_id, 'min'] = x.min()

In [ ]:
X_test_scaled = scaler.transform(X_test)
submission['time_to_failure'] = lm.predict(X_test_scaled)
submission.to_csv('submission.csv')

In [ ]:
#Random Forest

In [ ]:
from sklearn import metrics

print('MAE:', metrics.mean_absolute_error(y_train, predictions))
print('MSE:', metrics.mean_squared_error(y_train, predictions))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_train, predictions)))

In [ ]:
plt.scatter(y_train,predictions)
plt.xlabel('Y Test')
plt.ylabel('Predicted Y')

In [ ]:
submission = pd.read_csv('sample_submission.csv', index_col='seg_id')

In [ ]:
X_test = pd.DataFrame(columns=X_train.columns, dtype=np.float64, index=submission.index)

In [ ]:
for seg_id in X_test.index:
    seg = pd.read_csv('test/' + seg_id + '.csv')
    
    x = seg['acoustic_data'].values
    
    X_test.loc[seg_id, 'ave'] = x.mean()
    X_test.loc[seg_id, 'std'] = x.std()
    X_test.loc[seg_id, 'max'] = x.max()
    X_test.loc[seg_id, 'min'] = x.min()

In [ ]:
X_test_scaled = scaler.transform(X_test)
submission['time_to_failure'] = lm.predict(X_test_scaled)
submission.to_csv('submission.csv')

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X = X_train[['ave', 'std', 'max', 'min']]

y = y_train['time_to_failure']
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.30)

In [ ]:
from sklearn.model_selection import cross_val_predict, GridSearchCV,cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, r2_score

In [ ]:
rfr = RandomForestRegressor(random_state=0, n_estimators = 600, criterion='mae')

In [ ]:
cv_scores = cross_val_score(rfr, X, y, cv = 5)

In [ ]:
rfr.fit(X_train,y_train)

In [ ]:
predictions = rfr.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix

In [ ]:
print(rfr.score(X_test, y_test))

In [ ]:
plt.scatter(y_test,predictions)
plt.xlabel('Y Test')
plt.ylabel('Predicted Y')

In [ ]:
submission = pd.read_csv('sample_submission.csv', index_col='seg_id')

In [ ]:
X_test = pd.DataFrame(columns=X_train.columns, dtype=np.float64, index=submission.index)

In [ ]:
for seg_id in X_test.index:
    seg = pd.read_csv('test/' + seg_id + '.csv')
    
    x = seg['acoustic_data'].values
    
    X_test.loc[seg_id, 'ave'] = x.mean()
    X_test.loc[seg_id, 'std'] = x.std()
    X_test.loc[seg_id, 'max'] = x.max()
    X_test.loc[seg_id, 'min'] = x.min()


In [ ]:
X_test_scaled = scaler.transform(X_test)
submission['time_to_failure'] = rfr.predict(X_test)
submission.to_csv('submission.csv')

In [ ]:
#sample model

In [ ]:
scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)

In [ ]:
svm = NuSVR()
svm.fit(X_train_scaled, y_train.values.flatten())
y_pred = svm.predict(X_train_scaled)
print(y_pred)

In [ ]:
score = mean_absolute_error(y_train.values.flatten(), y_pred)
print(f'Score: {score:0.3f}')

In [ ]:
svm.score(X_train,y_train)

In [ ]:
submission = pd.read_csv('sample_submission.csv', index_col='seg_id')

In [ ]:
X_test = pd.DataFrame(columns=X_train.columns, dtype=np.float64, index=submission.index)

In [ ]:
print("There are {} files in test folder".format(len(os.listdir('test' ))))

In [ ]:
for seg_id in X_test.index:
    seg = pd.read_csv('test/' + seg_id + '.csv')
    
    x = seg['acoustic_data'].values
    
    X_test.loc[seg_id, 'ave'] = x.mean()
    X_test.loc[seg_id, 'std'] = x.std()
    X_test.loc[seg_id, 'max'] = x.max()
    X_test.loc[seg_id, 'min'] = x.min()

In [ ]:
X_test_scaled = scaler.transform(X_test)
submission['time_to_failure'] = svm.predict(X_test_scaled)
submission.to_csv('submission.csv')